In [42]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

import random

[nltk_data] Downloading package punkt to /home/sundeep/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sundeep/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [43]:


words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))





47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']
Training data created
Instructions f

/home/sundeep/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [44]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
opt = Adam(lr=0.001, decay=1e-6)



In [45]:


model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
47/47 [==============================] - 0s 5ms/sample - loss: 2.2274 - acc: 0.1489
Epoch 2/200
47/47 [==============================] - 0s 561us/sample - loss: 2.1356 - acc: 0.1277
Epoch 3/200
47/47 [==============================] - 0s 484us/sample - loss: 2.0849 - acc: 0.2766
Epoch 4/200
47/47 [==============================] - 0s 537us/sample - loss: 2.0184 - acc: 0.2979
Epoch 5/200
47/47 [==============================] - 0s 583us/sample - loss: 1.8940 - acc: 0.4468
Epoch 6/200
47/47 [==============================] - 0s 594us/sample - loss: 1.6198 - acc: 0.5532
Epoch 7/200
47/47 [==============================] - 0s 542us/sample - loss: 1.5650 - acc: 0.5745
Epoch 8/200
47/47 [==============================] - 0s 484us/sample - loss: 1.3185 - acc: 0.6383
Epoch 9/200
47/47 [==============================] - 0s 562us/sample - loss: 1.2839 - acc: 0.6809
Epoch 10/200
47/47 [==============================] - 0s 606us/sample - loss: 1.2141 - acc: 0.6596
Epoch 11/200
47/47 [=

47/47 [==============================] - 0s 500us/sample - loss: 0.1320 - acc: 0.9787
Epoch 84/200
47/47 [==============================] - 0s 539us/sample - loss: 0.0612 - acc: 1.0000
Epoch 85/200
47/47 [==============================] - 0s 510us/sample - loss: 0.0664 - acc: 0.9787
Epoch 86/200
47/47 [==============================] - 0s 569us/sample - loss: 0.0860 - acc: 0.9574
Epoch 87/200
47/47 [==============================] - 0s 535us/sample - loss: 0.0991 - acc: 0.9787
Epoch 88/200
47/47 [==============================] - 0s 500us/sample - loss: 0.0758 - acc: 0.9787
Epoch 89/200
47/47 [==============================] - 0s 508us/sample - loss: 0.0411 - acc: 1.0000
Epoch 90/200
47/47 [==============================] - 0s 535us/sample - loss: 0.0548 - acc: 1.0000
Epoch 91/200
47/47 [==============================] - 0s 554us/sample - loss: 0.0896 - acc: 0.9787
Epoch 92/200
47/47 [==============================] - 0s 522us/sample - loss: 0.0577 - acc: 0.9787
Epoch 93/200
47/47 [===

47/47 [==============================] - 0s 527us/sample - loss: 0.1162 - acc: 0.9362
Epoch 166/200
47/47 [==============================] - 0s 550us/sample - loss: 0.0130 - acc: 1.0000
Epoch 167/200
47/47 [==============================] - 0s 528us/sample - loss: 0.0907 - acc: 0.9574
Epoch 168/200
47/47 [==============================] - 0s 518us/sample - loss: 0.0153 - acc: 1.0000
Epoch 169/200
47/47 [==============================] - 0s 536us/sample - loss: 0.0211 - acc: 1.0000
Epoch 170/200
47/47 [==============================] - 0s 566us/sample - loss: 0.0331 - acc: 0.9787
Epoch 171/200
47/47 [==============================] - 0s 517us/sample - loss: 0.0253 - acc: 1.0000
Epoch 172/200
47/47 [==============================] - 0s 529us/sample - loss: 0.0062 - acc: 1.0000
Epoch 173/200
47/47 [==============================] - 0s 547us/sample - loss: 0.0939 - acc: 0.9787
Epoch 174/200
47/47 [==============================] - 0s 514us/sample - loss: 0.0619 - acc: 0.9787
Epoch 175/200


In [47]:
model.save('chatbot_model2.h5', hist)

print("model created")

model created
